# **SnP Market Pre-processing**

In [1]:
#SnP 데이터
import pandas as pd

SnP500 = pd.read_csv('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/SnP500구성종목_wikipedia.csv')

columns_to_read = ['pricingDate', 'tickerSymbol', 'ISOCode', 'priceClose', 'divAdjFactor']
market1 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market1.parquet', columns=columns_to_read)
market2 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market2.parquet', columns=columns_to_read)
market3 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market3.parquet', columns=columns_to_read)
market4 = pd.read_parquet('/Users/jaekyunpark/Library/CloudStorage/OneDrive-경희대학교/JaekyunPark/S&P data/market4.parquet', columns=columns_to_read)

In [2]:
# 'ISOCode' 열에 있는 고유한 값들을 확인
unique_iso_codes = market1['ISOCode'].unique()

# 결과 출력
print(unique_iso_codes)

['GBP' 'CHF' 'USD' 'CAD' 'EUR' 'JPY' 'KWD' 'SEK' 'SGD' 'HKD' 'TWD' 'BRL'
 'INR' 'ILS' 'ARS' 'CLP' 'CNY' 'MXN' 'AUD' 'RUB' 'MYR' 'THB' 'PHP' 'DKK'
 'KRW' 'PLN' 'RON' 'LKR' 'EGP' 'NOK' 'NZD' 'NGN' 'BDT' 'KZT' 'VES' 'HUF'
 'PEN' 'COP' 'ZAR' 'IDR' 'TRY' 'TZS' 'UGX' 'JMD' 'PKR' 'JOD' 'SAR' 'KES'
 'TTD' 'ISK' 'MAD' 'BMD' 'CZK' 'MWK' 'UAH' 'BGN' 'BHD' 'TND' 'BWP' 'MUR'
 'GHS' 'XOF' 'OMR' 'AED' 'QAR' 'RSD']


In [3]:
# 'ISOCode'가 'USD'인 행들만 유지
market1_usd = market1[market1['ISOCode'] == 'USD']
market2_usd = market2[market2['ISOCode'] == 'USD'] 
market3_usd = market3[market3['ISOCode'] == 'USD'] 
market4_usd = market4[market4['ISOCode'] == 'USD']

In [4]:
#SnP500 구성 종목들만 필터링
snP500_symbols = SnP500['Symbol'].unique()
market1_filtered = market1_usd[market1_usd['tickerSymbol'].isin(snP500_symbols)]
market2_filtered = market2_usd[market2_usd['tickerSymbol'].isin(snP500_symbols)]
market3_filtered = market3_usd[market3_usd['tickerSymbol'].isin(snP500_symbols)]
market4_filtered = market4_usd[market4_usd['tickerSymbol'].isin(snP500_symbols)]

#데이터프레임 병합
merged_market_data = pd.concat([market1_filtered, market2_filtered, market3_filtered, market4_filtered])

In [5]:
#결측치 확인(없음)
missing_price_data = merged_market_data[merged_market_data['priceClose'].isna()]

print(missing_price_data)

Empty DataFrame
Columns: [pricingDate, tickerSymbol, ISOCode, priceClose, divAdjFactor]
Index: []


In [6]:
# 수정종가 열 추가
# divadjfactor가 nan값인 경우 수정종가 계산이 안되는 문제가 있음. 이를 해결하기 위해 divadjfactor가 nan값인 경우 1로 대체
merged_market_data['divAdjFactor'].fillna(1, inplace=True)
merged_market_data['adjpriceClose'] = merged_market_data['priceClose'] * merged_market_data['divAdjFactor']
# 필요없는 열 삭제
merged_market_data = merged_market_data.drop(columns=['priceClose', 'divAdjFactor'])

In [7]:
# merged_market_data 종목 수
unique_ticker_symbols = merged_market_data['tickerSymbol'].nunique()
unique_ticker_symbols

499

In [8]:
# merged_market_data 재정렬 후 기간 맞춤

a = merged_market_data.pivot_table(index='pricingDate', columns='tickerSymbol', values='adjpriceClose' ,aggfunc='first')
a = a['2018-05-01':'2023-10-31']
a

tickerSymbol,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.5741047281600000,42.3427968821600000,40.1917075479600000,79.9409788993090000,NaN,53.9217798579020000,26.8766660000000000,140.3579726060400000,224.0800000000000000,79.9928624810340000,...,180.3430595460000000,39.8414884100250000,58.3266158807400000,47.8871695235040000,65.8279735925650000,78.5558930777760000,108.7708219120400000,136.7800000000000000,46.4626057460250000,80.5803988305000000
2018-05-02,63.2573859093150000,41.2490715014400000,41.9671780114920000,78.6095429815190000,NaN,53.0325563546350000,26.3100000000000000,139.2400608968950000,221.1000000000000000,79.4887436087700000,...,179.5722421948000000,39.7475427781900000,58.2129187737600000,47.8398875630740000,64.4341455315600000,72.7167892986800000,109.2451538417120000,133.1200000000000000,46.5807667778950000,77.1987274167500000
2018-05-03,63.6700801278100000,41.1221212340350000,42.0432356484840000,78.5390551976360000,NaN,53.1058943755230000,25.9833330000000000,140.1824327508850000,226.0500000000000000,79.1376608227290000,...,180.1268546792000000,39.6962997062800000,58.0158437883280000,46.5632746314640000,65.1357368374350000,74.9563213381780000,107.1246110972960000,133.5400000000000000,45.9562013237250000,77.6624994963500000
2018-05-04,64.3035177655000000,41.8545266229100000,43.6927356507480000,78.4529034617790000,NaN,53.8484418370140000,26.2766660000000000,141.7622914472800000,228.5100000000000000,81.2621617844130000,...,180.8694713956000000,39.6194350984150000,58.2887168450800000,47.2725040379140000,65.5566916209600000,74.7387149861620000,108.0918762087840000,134.6700000000000000,46.8086487679300000,78.8026058587000000
2018-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.6700000000000000,12.7300000000000000,173.6600000000000000,147.6900000000000000,127.410000,95.6500000000000000,80.4600000000000000,308.1121020102510000,518.4200000000000000,173.7900000000000000,...,88.2100000000000000,56.6300000000000000,111.5000000000000000,33.3000000000000000,91.2200000000000000,123.5300000000000000,109.6900000000000000,230.9400000000000000,33.3300000000000000,170.9900000000000000
2023-10-05,110.3500000000000000,12.8500000000000000,174.9100000000000000,147.4500000000000000,124.990000,96.2000000000000000,81.6400000000000000,308.3809868078940000,516.4400000000000000,172.1000000000000000,...,89.4900000000000000,56.4800000000000000,108.9900000000000000,32.1800000000000000,90.1400000000000000,121.8400000000000000,109.3000000000000000,222.5400000000000000,33.4000000000000000,172.0000000000000000
2023-10-06,110.6400000000000000,12.7600000000000000,177.4900000000000000,148.2400000000000000,126.360000,96.8800000000000000,82.1800000000000000,310.9005369487710000,526.6800000000000000,173.9700000000000000,...,92.4100000000000000,57.3500000000000000,107.1700000000000000,32.4800000000000000,90.9700000000000000,119.4600000000000000,110.9100000000000000,223.8500000000000000,33.6900000000000000,175.5800000000000000


In [9]:
# 각 column별 nan값 확인
# nan값은 평균 178개를 고정적으로 가지고 있는 column이 많음. (480종목)

a.isnull().sum()

tickerSymbol
A       178
AAL     178
AAPL    178
ABBV    178
ABNB    837
       ... 
YUM     178
ZBH     178
ZBRA    178
ZION    178
ZTS     178
Length: 499, dtype: int64

In [10]:
#nan값이 178이상인 종목들을 drop

nan_counts = a.isna().sum()

columns_to_drop = nan_counts[nan_counts > 178].index.tolist()

a_filtered = a.drop(columns=columns_to_drop)

a_filtered

tickerSymbol,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.5741047281600000,42.3427968821600000,40.1917075479600000,79.9409788993090000,53.9217798579020000,26.8766660000000000,140.3579726060400000,224.0800000000000000,79.9928624810340000,38.8012811952720000,...,180.3430595460000000,39.8414884100250000,58.3266158807400000,47.8871695235040000,65.8279735925650000,78.5558930777760000,108.7708219120400000,136.7800000000000000,46.4626057460250000,80.5803988305000000
2018-05-02,63.2573859093150000,41.2490715014400000,41.9671780114920000,78.6095429815190000,53.0325563546350000,26.3100000000000000,139.2400608968950000,221.1000000000000000,79.4887436087700000,38.2842754498320000,...,179.5722421948000000,39.7475427781900000,58.2129187737600000,47.8398875630740000,64.4341455315600000,72.7167892986800000,109.2451538417120000,133.1200000000000000,46.5807667778950000,77.1987274167500000
2018-05-03,63.6700801278100000,41.1221212340350000,42.0432356484840000,78.5390551976360000,53.1058943755230000,25.9833330000000000,140.1824327508850000,226.0500000000000000,79.1376608227290000,37.5690841686400000,...,180.1268546792000000,39.6962997062800000,58.0158437883280000,46.5632746314640000,65.1357368374350000,74.9563213381780000,107.1246110972960000,133.5400000000000000,45.9562013237250000,77.6624994963500000
2018-05-04,64.3035177655000000,41.8545266229100000,43.6927356507480000,78.4529034617790000,53.8484418370140000,26.2766660000000000,141.7622914472800000,228.5100000000000000,81.2621617844130000,37.7931199916640000,...,180.8694713956000000,39.6194350984150000,58.2887168450800000,47.2725040379140000,65.5566916209600000,74.7387149861620000,108.0918762087840000,134.6700000000000000,46.8086487679300000,78.8026058587000000
2018-05-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.6700000000000000,12.7300000000000000,173.6600000000000000,147.6900000000000000,95.6500000000000000,80.4600000000000000,308.1121020102510000,518.4200000000000000,173.7900000000000000,73.4700000000000000,...,88.2100000000000000,56.6300000000000000,111.5000000000000000,33.3000000000000000,91.2200000000000000,123.5300000000000000,109.6900000000000000,230.9400000000000000,33.3300000000000000,170.9900000000000000
2023-10-05,110.3500000000000000,12.8500000000000000,174.9100000000000000,147.4500000000000000,96.2000000000000000,81.6400000000000000,308.3809868078940000,516.4400000000000000,172.1000000000000000,73.1300000000000000,...,89.4900000000000000,56.4800000000000000,108.9900000000000000,32.1800000000000000,90.1400000000000000,121.8400000000000000,109.3000000000000000,222.5400000000000000,33.4000000000000000,172.0000000000000000
2023-10-06,110.6400000000000000,12.7600000000000000,177.4900000000000000,148.2400000000000000,96.8800000000000000,82.1800000000000000,310.9005369487710000,526.6800000000000000,173.9700000000000000,72.8600000000000000,...,92.4100000000000000,57.3500000000000000,107.1700000000000000,32.4800000000000000,90.9700000000000000,119.4600000000000000,110.9100000000000000,223.8500000000000000,33.6900000000000000,175.5800000000000000


In [11]:
# NaN값이 178개 이상인 종목들을 drop한 후 저장
# 일단 최종적으로 SnP500 지수는 총 485종목을 볼 것임
# 5년의 기간에서 178일을 제거

a_filtered = a_filtered.dropna(axis=0)
a_filtered.to_csv('/Users/jaekyunpark/Documents/GitHub/S-PQuantTeam01/new_market.csv')
a_filtered

tickerSymbol,A,AAL,AAPL,ABBV,ABT,ACGL,ACN,ADBE,ADI,ADM,...,WYNN,XEL,XOM,XRAY,XYL,YUM,ZBH,ZBRA,ZION,ZTS
pricingDate,,,,,,,,,,,,,,,,,,,,,
2018-05-01,63.5741047281600000,42.3427968821600000,40.1917075479600000,79.9409788993090000,53.9217798579020000,26.8766660000000000,140.3579726060400000,224.0800000000000000,79.9928624810340000,38.8012811952720000,...,180.3430595460000000,39.8414884100250000,58.3266158807400000,47.8871695235040000,65.8279735925650000,78.5558930777760000,108.7708219120400000,136.7800000000000000,46.4626057460250000,80.5803988305000000
2018-05-02,63.2573859093150000,41.2490715014400000,41.9671780114920000,78.6095429815190000,53.0325563546350000,26.3100000000000000,139.2400608968950000,221.1000000000000000,79.4887436087700000,38.2842754498320000,...,179.5722421948000000,39.7475427781900000,58.2129187737600000,47.8398875630740000,64.4341455315600000,72.7167892986800000,109.2451538417120000,133.1200000000000000,46.5807667778950000,77.1987274167500000
2018-05-03,63.6700801278100000,41.1221212340350000,42.0432356484840000,78.5390551976360000,53.1058943755230000,25.9833330000000000,140.1824327508850000,226.0500000000000000,79.1376608227290000,37.5690841686400000,...,180.1268546792000000,39.6962997062800000,58.0158437883280000,46.5632746314640000,65.1357368374350000,74.9563213381780000,107.1246110972960000,133.5400000000000000,45.9562013237250000,77.6624994963500000
2018-05-04,64.3035177655000000,41.8545266229100000,43.6927356507480000,78.4529034617790000,53.8484418370140000,26.2766660000000000,141.7622914472800000,228.5100000000000000,81.2621617844130000,37.7931199916640000,...,180.8694713956000000,39.6194350984150000,58.2887168450800000,47.2725040379140000,65.5566916209600000,74.7387149861620000,108.0918762087840000,134.6700000000000000,46.8086487679300000,78.8026058587000000
2018-05-07,64.6778218241350000,41.5413028506240000,44.0088502044960000,78.0299767584810000,54.3801424884520000,26.3766660000000000,140.7090523163500000,230.9900000000000000,81.4692106069500000,37.6466350304560000,...,180.3994608156000000,39.3973817868050000,58.9254206441680000,44.3693916675120000,65.5005643164900000,75.3552663168740000,108.8080244163280000,137.4800000000000000,46.8508491364550000,79.3726590398750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-10-04,111.6700000000000000,12.7300000000000000,173.6600000000000000,147.6900000000000000,95.6500000000000000,80.4600000000000000,308.1121020102510000,518.4200000000000000,173.7900000000000000,73.4700000000000000,...,88.2100000000000000,56.6300000000000000,111.5000000000000000,33.3000000000000000,91.2200000000000000,123.5300000000000000,109.6900000000000000,230.9400000000000000,33.3300000000000000,170.9900000000000000
2023-10-05,110.3500000000000000,12.8500000000000000,174.9100000000000000,147.4500000000000000,96.2000000000000000,81.6400000000000000,308.3809868078940000,516.4400000000000000,172.1000000000000000,73.1300000000000000,...,89.4900000000000000,56.4800000000000000,108.9900000000000000,32.1800000000000000,90.1400000000000000,121.8400000000000000,109.3000000000000000,222.5400000000000000,33.4000000000000000,172.0000000000000000
2023-10-06,110.6400000000000000,12.7600000000000000,177.4900000000000000,148.2400000000000000,96.8800000000000000,82.1800000000000000,310.9005369487710000,526.6800000000000000,173.9700000000000000,72.8600000000000000,...,92.4100000000000000,57.3500000000000000,107.1700000000000000,32.4800000000000000,90.9700000000000000,119.4600000000000000,110.9100000000000000,223.8500000000000000,33.6900000000000000,175.5800000000000000
